In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def create_coordinate_arrows(size=1.0, origin=[20, 0, 0]):
    """
    좌표축을 화살표로 생성하는 함수
    
    Args:
        size: float - 화살표 크기
        origin: list - 좌표축 원점 위치 [x, y, z]
    
    Returns:
        list: 화살표 geometry 리스트
    """
    geometries = []
    
    # 화살표 생성을 위한 실린더와 원뿔 생성
    for i, color in enumerate([[1,0,0], [0,1,0], [0,0,1]]):  # RGB for XYZ
        # 실린더 (화살표 몸통) 생성
        cylinder = o3d.geometry.TriangleMesh.create_cylinder(
            radius=size/30,
            height=size*0.8
        )
        # 실린더를 중심이 원점에 오도록 이동
        cylinder.translate([0, 0, size*0.4])
        
        # 원뿔 (화살표 머리) 생성
        cone = o3d.geometry.TriangleMesh.create_cone(
            radius=size/15,
            height=size*0.2
        )
        # 원뿔을 실린더 위에 배치
        cone.translate([0, 0, size*0.8])
        
        # 화살표 합치기
        arrow = cylinder + cone
        
        # 각 축 방향으로 회전
        if i == 0:  # X축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([0, np.pi/2, 0])
            )
        elif i == 1:  # Y축
            arrow.rotate(
                o3d.geometry.get_rotation_matrix_from_xyz([-np.pi/2, 0, 0])
            )
        
        # 원점으로 이동
        arrow.translate(origin)
        
        # 색상 지정
        arrow.paint_uniform_color(color)
        geometries.append(arrow)
    
    return geometries


In [3]:
def show(meshes):
    """
    여러 메쉬를 동시에 시각화하는 함수
    
    Args:
        meshes: list of o3d.geometry.TriangleMesh or single mesh
    """
    # 단일 메쉬인 경우 리스트로 변환
    if not isinstance(meshes, list):
        meshes = [meshes]
    
    # 모든 메쉬의 바운딩 박스를 고려하여 좌표축 크기 계산
    max_size = 0
    for mesh in meshes:
        if isinstance(mesh, o3d.geometry.TriangleMesh):
            mesh_bbox = mesh.get_axis_aligned_bounding_box()
            mesh_size = mesh_bbox.get_extent()
            max_size = max(max_size, max(mesh_size))
    
    axis_size = max_size * 0.2

    # 좌표축 화살표 생성
    coordinate_arrows = create_coordinate_arrows(size=axis_size)

    # 모든 메쉬와 좌표축 함께 시각화
    geometries = meshes
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    
    # 각 geometry 추가
    for g in geometries:
        # 메쉬인 경우 반투명 설정
        if isinstance(g, o3d.geometry.TriangleMesh):
            # 반투명 회색 설정 (알파값 0.5)
            pcd = g.sample_points_uniformly(number_of_points=50000)
            #pcd.paint_uniform_color([0.7, 0.7, 0.7])
            vis.add_geometry(pcd)
        else:
            vis.add_geometry(g)

    for g in coordinate_arrows:
        vis.add_geometry(g)
    # 렌더링 옵션 설정
    opt = vis.get_render_option()
    opt.mesh_show_back_face = True
    opt.mesh_show_wireframe = False
    opt.background_color = np.asarray([0, 0, 0])
    
    # 카메라 설정
    ctr = vis.get_view_control()
    ctr.set_zoom(0.8)
    
    vis.run()
    vis.destroy_window()

In [20]:
import open3d as o3d
import numpy as np

# 1. 메쉬 로드
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()

# 2. OBB 추가
obb = mesh.get_oriented_bounding_box()
obb.color = [1, 0, 0]  # 빨간색

# 3. 시각화
o3d.visualization.draw_geometries([mesh, obb])

In [25]:
import open3d as o3d
import numpy as np

# 1. 메쉬 로드
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()

# 2. OBB 추가
obb = mesh.get_oriented_bounding_box()
obb.color = [1, 0, 0]  # 빨간색

# 3. PCA 분석
vertices = np.asarray(mesh.vertices)
center = vertices.mean(axis=0)
vertices_centered = vertices - center
covariance_matrix = np.cov(vertices_centered.T)
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
idx = eigenvalues.argsort()
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

# 화살표 기본 크기 설정
max_arrow_scale = np.max(obb.extent)

# eigenvalues를 0-1 사이로 정규화
normalized_eigenvalues = eigenvalues / np.max(eigenvalues)

print("Eigenvalues:", eigenvalues)
print("Normalized Eigenvalues:", normalized_eigenvalues)

# PCA 방향을 화살표로 시각화
arrows = []
colors = [[1,0,0], [0,1,0], [0,0,1]]  # RGB for each principal component
for i in range(3):
    # 각 주성분의 크기에 따라 화살표 크기 조절
    arrow_scale = max_arrow_scale * normalized_eigenvalues[i]
    
    # 실린더 (화살표 몸통) 생성 - 더 얇게
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(
        radius=max_arrow_scale/40,  # 더 얇게 설정
        height=arrow_scale*0.8
    )
    # 실린더를 중심이 원점에 오도록 이동
    cylinder.translate([0, 0, arrow_scale*0.4])
    
    # 원뿔 (화살표 머리) 생성
    cone = o3d.geometry.TriangleMesh.create_cone(
        radius=max_arrow_scale/20,  # 실린더 반지름의 2배
        height=arrow_scale*0.2
    )
    # 원뿔을 실린더 위에 배치
    cone.translate([0, 0, arrow_scale*0.8])
    
    # 화살표 합치기
    arrow = cylinder + cone
    
    # 주성분 방향으로 회전
    direction = eigenvectors[:, i]
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, direction)
    rotation_angle = np.arccos(np.dot(z_axis, direction))
    
    if np.any(rotation_axis):
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(
            rotation_axis * rotation_angle
        )
        arrow.rotate(rotation_matrix)
    
    # 중심으로 이동
    arrow.translate(center)
    
    # 색상 지정
    arrow.paint_uniform_color(colors[i])
    arrows.append(arrow)

# 4. 시각화
o3d.visualization.draw_geometries([mesh, obb] + arrows)

Eigenvalues: [ 27.93309714 191.25079208 305.64572591]
Normalized Eigenvalues: [0.09139044 0.62572703 1.        ]


In [7]:
import open3d as o3d
import numpy as np

# 1. 메쉬 로드
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()

# 2. 메쉬에 색상 지정
mesh.paint_uniform_color([0.7, 0.7, 0.7])

# 3. 가장 기본적인 시각화
o3d.visualization.draw_geometries([mesh])

In [1]:
import open3d as o3d
import numpy as np

# 1. 메쉬 로드
mesh = o3d.io.read_triangle_mesh("assets/data/ios_with_smilearch.stl")
mesh.compute_vertex_normals()
mesh.paint_uniform_color([0.7, 0.7, 0.7])

# 2. PCA 분석
vertices = np.asarray(mesh.vertices)
center = vertices.mean(axis=0)
vertices_centered = vertices - center
covariance_matrix = np.cov(vertices_centered.T)
eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)
idx = eigenvalues.argsort()
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

print("Eigenvalues:", eigenvalues)
print("Eigenvectors:\n", eigenvectors)

# 3. 좌표축 생성
max_size = np.max(vertices.max(axis=0) - vertices.min(axis=0))
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=max_size * 0.2,
    origin=center
)

# 4. 주성분 방향을 보여주는 선 생성
lines = []
colors = []
points = []
for i in range(3):
    direction = eigenvectors[:, i] * eigenvalues[i] * 0.5  # 크기 조절
    points.append(center - direction)
    points.append(center + direction)
    colors.append([1, 0, 0] if i == 0 else [0, 1, 0] if i == 1 else [0, 0, 1])
    colors.append([1, 0, 0] if i == 0 else [0, 1, 0] if i == 1 else [0, 0, 1])

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector([[2*i, 2*i+1] for i in range(3)])
line_set.colors = o3d.utility.Vector3dVector(colors)

# 5. 시각화
o3d.visualization.draw_geometries([mesh, coordinate_frame, line_set])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Eigenvalues: [ 27.93309714 191.25079208 305.64572591]
Eigenvectors:
 [[ 0.06756749  0.19381579 -0.97870837]
 [ 0.98886171 -0.1433974   0.03987115]
 [-0.13261657 -0.97050123 -0.20134601]]
